**Python version:**  3.x

### Variabili da Inizializzare

In [1]:
# set the output file name
outFileName = 'Out_File_Name'


# set the path of the directory where your modules are
modulesPath = 'C:\\Users\\loren\\Documents\\Progetti\\Scuola\\AdvDataMining\\__Repo__'

#### Moduli

In [2]:
import os
import sys

if os.path.isdir(modulesPath) and modulesPath not in sys.path:
    sys.path.append(modulesPath)

modulesList = os.listdir(modulesPath)
print('Available Files and Modules for the project:', modulesList,sep='\n')


from collections import defaultdict
import itertools

%matplotlib inline

import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

from sklearn.model_selection import RandomizedSearchCV

import data_understanding
import data_preparation
import file_handling

Available Files and Modules for the project:
['datatest.txt', 'datatest2.txt', 'datatraining.txt', 'data_preparation.py', 'data_understanding.py', 'file_handling.py', 'machine_learning.py', '__pycache__']


In [3]:
df = file_handling.setup_df('datatraining.txt')

In [4]:
X_train, X_test, y_train, y_test = file_handling.setup_df('datatraining.txt', 'Occupancy', 'datatest.txt')

In [5]:
del X_train['date']
del X_test['date']

### Variabili da Inizializzare

In [6]:
#seleziona il modello
model = DecisionTreeClassifier()

# per conoscere tutti i possibili iper-parametri del modello:
model.get_params(deep=False)

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [7]:
#setta il range in cui devono essere cercati gli iper-parametri migliori peril modello

params_domain_dic = {'min_samples_leaf': range(1,150)}

## Ricerca dei migliori Iper-Parametri

In [8]:
clf = RandomizedSearchCV(model, param_distributions=params_domain_dic, n_iter=100, scoring='f1_macro', random_state=0)

clf.fit(X_train, y_train)

best_params = clf.best_params_
best_params

{'min_samples_leaf': 134}

## Test del modello

In [9]:
y_pred = clf.predict(X_test)

print('Accuracy: %s' % accuracy_score(y_test, y_pred))
print()
print('F1-score: %s' % f1_score(y_test, y_pred, average=None))
print()
print('Average F1-score: %s' % clf.score(X_test, y_test))
print()
print(classification_report(y_test, y_pred))

Accuracy: 0.9786116322701689

F1-score: [0.98290855 0.97142857]

Average F1-score: 0.977168558577854

             precision    recall  f1-score   support

          0       1.00      0.97      0.98      1693
          1       0.95      1.00      0.97       972

avg / total       0.98      0.98      0.98      2665



## Validazione del Modello

In [10]:
#riproduci il modello usando i parametri migliori
best_model = DecisionTreeClassifier(min_samples_leaf=best_params['min_samples_leaf'])

In [11]:
best_model_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='f1_macro')

print("F1 Score at a 95 percent confidence interval: %0.2f (+/- %0.2f)" % (
    best_model_scores.mean(), best_model_scores.std() * 2))

F1 Score at a 95 percent confidence interval: 0.97 (+/- 0.06)


## Output Files

In [12]:
#df.to_csv(path_or_buf = modulesPath + '\\' + outFileName + '.csv', sep=',', encoding='utf-8', index=False)